In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import mdtraj as md
#import mcg
import h5py
import pandas as pd
import sys
sys.path.append('../')
import matplotlib_config as cfg
import fnmatch
from matplotlib import cm
import pickle
from matplotlib.colors import PowerNorm

In [88]:
AA = True

In [89]:
data_base = '/leonardo_scratch/fast/L-AUT_Coretti/ClathrateTPSData/1wS'
cage_base_path = '/leonardo_work/L-AUT_Coretti/CODE/ClathrateTPS/1wS_cagedata'


In [95]:
if AA == True:
    datafolders =['../AlwaysAcceptingClathrates/data_mu175_chain0_batch_0/', '../AlwaysAcceptingClathrates/data_mu175_chain0_batch_1/', '../AlwaysAcceptingClathrates/data_mu175_chain0_batch_2/', '../AlwaysAcceptingClathrates/data_mu175_chain0_batch_3/']
else:
    datafolders = ['../ClathrateTPS/data_mu175_chain1_batch_1/', '../ClathrateTPS/data_mu175_chain1_batch_2/', '../ClathrateTPS/data_mu175_chain1_batch_3/', '../ClathrateTPS/data_mu175_chain1_batch_4/']

gro_folder = 'gro_files'

In [96]:
print(datafolders)

['../AlwaysAcceptingClathrates/data_mu175_chain0_batch_0/', '../AlwaysAcceptingClathrates/data_mu175_chain0_batch_1/', '../AlwaysAcceptingClathrates/data_mu175_chain0_batch_2/', '../AlwaysAcceptingClathrates/data_mu175_chain0_batch_3/']


In [97]:
def build_pd(run_folder_path):
    logfile_path = os.path.join(run_folder_path, 'log.txt')
    weights = np.load(os.path.join(run_folder_path, 'trajectory_weights.npy'))[1:]
    
    # Load the log file
    df = pd.read_fwf(
        logfile_path,
        header=0,
        widths=[21] * 7,
        strip=True
    )
    df.columns = df.columns.str.strip()
    
    # Initialize weights column with zeros
    df['Weights'] = 0.0
    
    # Get successful rows
    successful_indices = df[df['Successful'] == True].index
    
    # Sanity check
    assert len(successful_indices) == len(weights), "Mismatch between successful trajectories and weights"
    
    # Assign weights to successful trajectories
    for idx, weight in zip(successful_indices, weights):
        df.at[idx, 'Weights'] = weight

    return df

In [98]:
total_generated = 0
total_accepted = 0

for batch in datafolders:
    print(batch)

    all_generated = 0
    all_accepted = 0
    
    for runfolder in os.listdir(batch):
        if not runfolder.startswith('.') and not runfolder.endswith('lock'):
            runfolder_path = os.path.join(batch, runfolder)
            df = build_pd(runfolder_path)
            #display(df)
            num_generated = len(df)
            all_generated += num_generated

            accepted_num = df['Index'].values[-1]
            all_accepted  += accepted_num

    total_generated += all_generated
    total_accepted += all_accepted
    
    print(f'For batch {batch.split("/")[-2]} we generated {all_generated} trajectories and accepted {all_accepted}.')

print(f'In total {total_generated} were generated and {total_accepted} were accepted, {total_generated-total_accepted} rejected.')

../AlwaysAcceptingClathrates/data_mu175_chain0_batch_0/
For batch data_mu175_chain0_batch_0 we generated 236 trajectories and accepted 236.
../AlwaysAcceptingClathrates/data_mu175_chain0_batch_1/
For batch data_mu175_chain0_batch_1 we generated 249 trajectories and accepted 249.
../AlwaysAcceptingClathrates/data_mu175_chain0_batch_2/
For batch data_mu175_chain0_batch_2 we generated 247 trajectories and accepted 247.
../AlwaysAcceptingClathrates/data_mu175_chain0_batch_3/
For batch data_mu175_chain0_batch_3 we generated 241 trajectories and accepted 241.
In total 973 were generated and 973 were accepted, 0 rejected.
